<a href="https://colab.research.google.com/github/jko6422/testfile/blob/main/test_06_15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 특수문자 손글씨 인식 프로그램

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
cd /content/drive/MyDrive/colab/Minst_test/

데이터 세트 제작 파트

In [ ]:
#필요 함수 import
import cv2 
import numpy as np
from google.colab.patches import cv2_imshow

"""
1. 데이터 세트 만들기 파트.
원활한 이미지 인식을 위한 준비 작업.
"""

#이미지 파일 경로 지정. 이미지는 camscanner로 보정 후 드라이브에 업로드한다.
filename = '/content/drive/MyDrive/colab/Minst_test/data/testfile1.jpg'

#그레이 스케일로 변환 
src = cv2.imread(filename,cv2.IMREAD_GRAYSCALE) 
print("기존 사진 사이즈")
print(src.shape[1], src.shape[0])

#이미지의 크기를 비율 맞춰 조정.
#기본 수치 744. 안되면 740으로 해보고 그래도 안되면 임의조정.
changed_width = 744
n = (src.shape[1] / changed_width)

changed_height = (src.shape[0] / n)
src = cv2.resize(src , ( changed_width,int(changed_height) ) ) 

print("현재 사진 사이즈")
print(src.shape[1], src.shape[0], "\n")

#양 옆의 노이즈를 제거. 불필요할 시 각주처리할 것.
#src = src[0:src.shape[0]-10, 15:src.shape[1]-25]

#영상 이진화 
ret , binary = cv2.threshold(src,170,255,cv2.THRESH_BINARY_INV) 

#노이즈 제거(camscanner에 의해 어느 정도 보정이 되었으므로 각주 처리)
#binary = cv2.morphologyEx(binary , cv2.MORPH_OPEN , cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2,2)), iterations = 2) 

#외곽선 검출
contours , hierarchy = cv2.findContours(binary , cv2.RETR_EXTERNAL , cv2.CHAIN_APPROX_SIMPLE)
 
#이진화 이미지를 color로 복사한다.
color = cv2.cvtColor(binary, cv2.COLOR_GRAY2BGR)

#초록색으로 외곽선을 그려준다.
cv2.drawContours(color , contours , -1 , (0,255,0),3)

"""
검출한 외곽선에 사각형을 그려서 배열에 추가. 이때 크기가 작은 노이즈는 1차로 거른다.
boundingRect : 외각선을 감싸는 최소 크기 사각형 반환. x, y는 좌표. w는 폭, h는 넓이.
append : 리스트에 요소 추가
"""

bR_arr = [] 

for i in range(len(contours)) : 
  bin_tmp = binary.copy() 
  x,y,w,h = cv2.boundingRect(contours[i])
  if x and y > 0 : 
      bR_arr.append([x,y,w,h])

"""
!와 ?의 점까지 인식해보자 프로젝트.
막대기 좌표에 점의 좌표를 더하기!
"""

# 1) 네모가 생길 위치 좌표 저장. 물음표와 느낌표를 찾을때의 x축 범위.
erase_dot = []
count = 0
county = 0

# x를 기준으로 정렬
bR_arr = sorted(bR_arr, key=lambda num : num[0], reverse = False)

# 네모가 그려질 위치를 미리 파악한다.
for x,y,w,h in bR_arr : 
  tmp_y = bin_tmp[y-2:y+h+2,x-2:x+w+2].shape[0] #y축 계산
  tmp_x = bin_tmp[y-2:y+h+2,x-2:x+w+2].shape[1] #x축 계산
  # %는 크기를 넓게 세팅.
  # ^ 와 * 은 크기가 작으므로 조건없이 별도로 처리하게 세팅.
  # 그 이외는 글자 크기에 맞게 세팅.
  if (tmp_x and tmp_y > 20) or county == 5 or county == 7: 
      erase_dot.append([x,y]) 
      count += 1

      if count == 10 : 
          count = 0
          county += 1

#100개 들어왔으면 성공. 아니면 이미지 크기를 다시 조정할 것.
print("현재 인식 수치")
print(len(erase_dot))

# 2) 막대기와 점을 연결하자!

# 적용된 문자 수 판별
count = 0

# 1은 느낌표 범위
erase_dot = sorted(erase_dot, key=lambda num : num[0], reverse = False)
erase_dot_x_1 = erase_dot[9][0]
print("느낌표 적용범위", erase_dot_x_1)

# 느낌표. 2번 반복!
bR_arr = sorted(bR_arr, key=lambda num : num[0], reverse = False)

for n in range(0, 2) : 
    for i in range(0, len(bR_arr)-1) :
        # y축 차이 계산. 배열수 감소로 에러 시 break하고 다음 i로 이동하게 설정.
        for j in range(i+1, len(bR_arr)-1) :
            try : 
                y_difference = int(bR_arr[i][1]) - int(bR_arr[j][1])
            except : 
                break

            if int(bR_arr[i][0] ) <= erase_dot_x_1 and int(bR_arr[j][0]) <= erase_dot_x_1 : 
                # y의 길이 차가 별로 안 날 것.
                if abs(y_difference) < 70 : 

                    # 높이 비교로 막대기 부분 점 부분 판정.
                    if (int(bR_arr[i][3]) < 10) and ( int(bR_arr[j][3]) > 10) :
                        # [i]가 점, [j]가 막대기
                        if int(bR_arr[i][3] + bR_arr[i][1] - bR_arr[j][1]) > 0 :
                            bR_arr[j][3] = bR_arr[i][3] + bR_arr[i][1] - bR_arr[j][1] 

                            if bR_arr[i][0] < bR_arr[j][0] : 
                                bR_arr[j][0] = bR_arr[i][0]
                                bR_arr[j][2] += bR_arr[i][2]

                            del bR_arr[i]
                            count += 1

                    elif (int(bR_arr[i][3]) > 10) and ( int(bR_arr[j][3]) < 10) : 
                        # [i]가 막대기, [j]가 점
                        if int(bR_arr[j][3] + bR_arr[j][1] - bR_arr[i][1]) > 0:
                            bR_arr[i][3] = bR_arr[j][3] + bR_arr[j][1] - bR_arr[i][1]

                            if bR_arr[j][0] < bR_arr[i][0] : 
                                bR_arr[i][0] = bR_arr[j][0]
                                bR_arr[i][2] += bR_arr[j][2]

                            del bR_arr[j]
                            count += 1

# 보정 수치.
length = len(erase_dot) - 100

# 2,3은 물음표 범위
for i in range (79+length,len(erase_dot)-1) : 
    if erase_dot[i+1][0] - erase_dot[i][0] >10 :
        erase_dot_x_2 = erase_dot[i+1][0]
        break

for i in range (89+length,len(erase_dot)-1) : 
    if erase_dot[i+1][0] - erase_dot[i][0] >10 :
        erase_dot_x_3 = erase_dot[i+1][0]
        break

print("물음표 적용범위", erase_dot_x_2, erase_dot_x_3)

#물음표. 2번 반복!
bR_arr = sorted(bR_arr, key=lambda num : num[0], reverse = False)

for n in range(0,2) : 
    for i in range(0, len(bR_arr)-1) :
        for j in range(i+1, len(bR_arr)-1) :
            try : 
                y_difference = int(bR_arr[i][1]) - int(bR_arr[j][1])
            except : 
                break

            if int(bR_arr[i][0] ) >= erase_dot_x_2  and int(bR_arr[j][0]) >= erase_dot_x_2  :
                if int(bR_arr[i][0] ) <= erase_dot_x_3  and int(bR_arr[j][0]) <= erase_dot_x_3  : 

                    # y의 길이 차가 별로 안 날 것.
                    if abs(y_difference) < 70 : 
                        # 높이 비교로 막대기 부분 점 부분 판정.

                        if (int(bR_arr[i][3]) < 10) and ( int(bR_arr[j][3]) > 10) :
                            # [i]가 점, [j]가 막대기
                        
                            if int(bR_arr[i][3] + bR_arr[i][1] - bR_arr[j][1]) > 0 :
                                bR_arr[j][3] = bR_arr[i][3] + bR_arr[i][1] - bR_arr[j][1]

                                if bR_arr[i][0] < bR_arr[j][0] : 
                                    bR_arr[j][0] = bR_arr[i][0]
                                    bR_arr[j][2] += bR_arr[i][2]

                                del bR_arr[i]
                                count += 1

                        elif (int(bR_arr[i][3]) > 10) and ( int(bR_arr[j][3]) < 10) :                        
                            # [i]가 막대기, [j]가 점

                            if int(bR_arr[j][3] + bR_arr[j][1] - bR_arr[i][1]) > 0:
                                bR_arr[i][3] = bR_arr[j][3] + bR_arr[j][1] - bR_arr[i][1]

                                if bR_arr[j][0] < bR_arr[i][0] : 
                                    bR_arr[i][0] = bR_arr[j][0]
                                    bR_arr[i][2] += bR_arr[j][2]

                                del bR_arr[j]
                                count += 1

print("느낌표 배열")
print(bR_arr[:10])

print("물음표 배열")
for i in range(0, len(bR_arr)-1) : 
    if int(bR_arr[i][0] ) >= erase_dot_x_2 -1 and int(bR_arr[i][0] ) <= erase_dot_x_3 +1:
        print(bR_arr[i])

print("느낌표 물음표 보정 적용 수 : ", count)

"""

 글자 @와 % 를 보정하자.
 직사각형을 그릴 시, 특수문자 전체만이 아닌 일부까지 인식하는 것을 막도록 노이즈 배열을 제거한다.
 큰 네모와 작은 네모가 있을 때 x 좌표와 y 좌표를 비교 후(x_big < x_small 이고 y_big < y_small 일 것. x와 y를 비교)
 작은 네모가 큰 네모 안에 있는지 비교하고 (x_big_1 > x_small_1 이고 y_big_1 > y_small_1일 것. x+w와 y+h를 비교)
 조건이 맞을 시 작은 네모로 판정된 배열을 제거한다.

"""
#y로 정렬한 뒤 시작.
bR_arr = sorted(bR_arr, key=lambda num : num[1], reverse = False)

for n in (0,2) : 
    for i in range(0, len(bR_arr)-1) :
        for j in range(i+1, len(bR_arr)-1) : 

        #베열 파악 중 오류 발생 시 j for문을 break 후 다음 i로 넘기도록 세팅.
        # i가 큰 네모, j가 작은 네모라 가정 후 판단
            try : 
                x_big = int(bR_arr[i][0]) - 10
                y_big = int(bR_arr[i][1]) - 10
                x_small = int(bR_arr[j][0]) 
                y_small = int(bR_arr[j][1])
            except : 
                break

            if x_big < x_small and y_big < y_small : 
                x_big_1 = int(bR_arr[i][0] + bR_arr[i][2]) + 10
                y_big_1 = int(bR_arr[i][1] + bR_arr[i][3]) + 10
                x_small_1 = int(bR_arr[j][0] + bR_arr[j][2])
                y_small_1 = int(bR_arr[j][1] + bR_arr[j][3])

                if x_big_1 > x_small_1 and y_big_1 > y_small_1 :
                    del bR_arr[j]

        # j가 큰 네모, i가 작은 네모라 가정 후 판단
            x_big = int(bR_arr[j][0]) - 10
            y_big = int(bR_arr[j][1]) - 10
            x_small = int(bR_arr[i][0]) 
            y_small = int(bR_arr[i][1])
            
            if x_big < x_small and y_big < y_small : 
                x_big_1 = int(bR_arr[j][0] + bR_arr[j][2]) + 10
                y_big_1 = int(bR_arr[j][1] + bR_arr[j][3]) + 10
                x_small_1 = int(bR_arr[i][0] + bR_arr[i][2])
                y_small_1 = int(bR_arr[i][1] + bR_arr[i][3])

                if x_big_1 > x_small_1 and y_big_1 > y_small_1 :
                    del bR_arr[i]

# 다시 x축 정렬 후 확인
print("최종적으로 남은 배열 수")
print(len(bR_arr))

bR_arr = sorted(bR_arr, key=lambda num : num[0], reverse = False)
print(bR_arr[40:60])

"""
# 데이터에 사각형그리기. 10개씩 리스트로 묶어서 저장 
# 순서는 !부터 위에서 아래로 10번(count). 그 뒤에 다음 이모티콘을 저장하는 방식.(county)
# countbox는 네모가 쳐진 갯수. 100개면 성공.
"""
digit_arr = [] 
digit_arr2 = [] 

count = 0
county = 0
countbox = 0

bR_arr = sorted(bR_arr, key=lambda num : num[0], reverse = False)

for x,y,w,h in bR_arr : 
  tmp_y = bin_tmp[y-2:y+h+2,x-2:x+w+2].shape[0] #y축 계산
  tmp_x = bin_tmp[y-2:y+h+2,x-2:x+w+2].shape[1] #x축 계산

  # ^ 와 * 은 크기가 작으므로 조건없이 별도로 처리하게 세팅.
  # %,는 크기를 넓게 세팅. !와 ?는 x축만 조금 보정.
  # 그 이외는 글자 크기에 맞게 세팅.
  if (tmp_x and tmp_y > 20) or county == 5 or county == 7: 
      if county == 0 or county == 8: 
          cv2.rectangle(color,(x-4,y-2),(x+w+4,y+h+2),(0,0,255),1) 
          digit_arr.append(bin_tmp[y-2:y+h+2,x-4:x+w+4]) 
      elif county == 4 : 
          cv2.rectangle(color,(x-10,y-10),(x+w+10,y+h+10),(0,0,255),1) 
          digit_arr.append(bin_tmp[y-10:y+h+10,x-10:x+w+10]) 
      else : 
          cv2.rectangle(color,(x-2,y-2),(x+w+2,y+h+2),(0,0,255),1) 
          digit_arr.append(bin_tmp[y-2:y+h+2,x-2:x+w+2]) 
      
      count += 1
      countbox += 1

      if count == 10 : 
          digit_arr2.append(digit_arr) 
          digit_arr = [] 
          count = 0
          county += 1

#결과 확인
cv2_imshow(color) 
k = cv2.waitKey(0) 
cv2.destroyAllWindows()

print(countbox)

In [ ]:
#리스트에 저장된 이미지를 32x32의 크기로 리사이즈해서 순서대로 저장 
for i in range(0,len(digit_arr2)) : 
  for j in range(len(digit_arr2[i])) :
     
    # !과 ?의 경우 비율 유지를 위한 마스크 생성
     if i == 0 or i==8: 
         width = digit_arr2[i][j].shape[1]
         height = digit_arr2[i][j].shape[0] 
         tmp = (height - width)/2 
         mask = np.zeros((height,height)) 
         mask[0:height,int(tmp):int(tmp)+width] = digit_arr2[i][j]
         digit_arr2[i][j] = cv2.resize(mask,(32,32)) 
     else:
         digit_arr2[i][j] = cv2.resize(digit_arr2[i][j],(32,32))

# 훈련용 이미지는 data/training
# 테스트용 이미지는 data/test

for i in range(0,len(digit_arr2)) : 
  for j in range(len(digit_arr2[i])) :
      cv2.imwrite('data/test/'+str(i)+'_'+str(j)+'.png',digit_arr2[i][j])

In [ ]:
#데이터 세트를 코랩에 인식시키기

import os 
import cv2
import numpy as np
from sklearn.preprocessing import * 
import matplotlib.pyplot as plt 

def createdataset(directory):       #sklearn사용을 위해 데이터세트를 생성
    files = os.listdir(directory)
    x = []
    y = []
    for file in files:
        attr_x = cv2.imread(directory+file, cv2.IMREAD_GRAYSCALE)
   #     ret , attr_x = cv2.threshold(attr_x,170,255,cv2.THRESH_BINARY) #영상 이진화
        attr_x = attr_x.reshape(32,32,1)
        attr_y = int(file[0])
        
        x.append(attr_x)
        y.append(attr_y)
        
    x = np.array(x)
    y = np.array(y)    
    return x , y

train_dir = 'data/training/'
train_images ,train_labels = createdataset(train_dir)

test_dir = 'data/test/'
test_images , test_labels = createdataset(test_dir)

print('data set shape :',train_images.shape,train_labels.shape,test_images.shape,test_labels.shape)

# 라벨 설정 및 전처리
class_names = ['!', '@', '#', '$', '%',
               '^', '&', '*', '?', '+']

train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
데이터 모델 제작 파트

In [ ]:
#예제 1
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models

# 모델 객체를 만듭니다
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

# 모델 구조를 출력합니다
model.summary()

model.compile(optimizer='adam',
              loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# 모델 훈련하기
model.fit(train_images, train_labels,  epochs=30,
          validation_data=(test_images,test_labels),
          batch_size=200,
          verbose=1)  # 콜백을 훈련에 전달합니다

loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("모델의 정확도: {:5.2f}%".format(100*acc))

# 옵티마이저의 상태를 저장하는 것과 관련되어 경고가 발생할 수 있습니다.
# 이 경고는 (그리고 이 노트북의 다른 비슷한 경고는) 이전 사용 방식을 권장하지 않기 위함이며 무시해도 좋습니다.

# SavedModel로 전체 모델을 저장합니다
!mkdir -p saved_model
model.save('saved_model/my_model.h5')

In [ ]:
# 3. 딥러닝 결과 확인
# 딥러닝에 필요한 케라스 함수 호출
from keras.models import load_model
from keras.utils import np_utils

# 필요 라이브러리 호출(PIL은 이미지파일 처리위함)
from PIL import Image
import numpy as np

# test.png는 그림판에서 그린 이미지 파일. 이미지가 있는 경로를 아래 url위치에 넣어주세요
# test.png 파일을 열어서 L(256단계 흑백이미지)로 변환
img = Image.open("test4.png").convert("L")

# 변환된 이미지 보기
plt.imshow(img)
plt.show

# 데이터를 모델에 적용할 수 있도록 가공
img = np.resize(img, (100, 32, 32, 1))
test_data = ((np.array(img) / 255) - 1) * -1

# 모델 불러오기
model = load_model('saved_model/my_model.h5')

# 클래스 예측 함수에 가공된 테스트 데이터 넣어 결과 도출
class_names = ['!', '@', '#', '$', '%', '^', '&', '*', '?', '+']
res = np.argmax(model.predict(test_data), axis=-1)
print("결과는 : ", class_names[res[0]])